In [ ]:
# Cell 2: config - Edit the Values
import time
import json
import requests
import pandas as pd
import plotly.express as px
import os
import gzip
from datetime import datetime

BASE_URL = "YOUR_API_URL"  # no trailing slash
API_KEY = os.getenv("XDR_API_KEY", "YOUR_API_KEY")
API_KEY_ID = os.getenv("XDR_API_KEY_ID", "YOUR_API_KEY_ID")
# Time range for the hunt (last 30 days)
DEFAULT_RELATIVE_TIME_MS = 30 * 24 * 60 * 60 * 1000
# Global polling settings for long running XQL queries
POLL_MAX_ATTEMPTS = 120        # times to poll before giving up
POLL_SLEEP_SECONDS = 60       # wait time between polls
POLL_MAX_ATTEMPT = POLL_MAX_ATTEMPTS



# ===========================================================
# Session with headers
# ===========================================================
SESSION = requests.Session()
SESSION.headers.update({
    "Authorization": API_KEY,
    "x-xdr-auth-id": API_KEY_ID,
    "Accept-Encoding": "gzip",
    "Content-Type": "application/json"
})
def start_xql_query(query, relative_time_ms=DEFAULT_RELATIVE_TIME_MS, tenants=None):
   
   url = f"{BASE_URL}/public_api/v1/xql/start_xql_query"
   timeframe = {"relativeTime": relative_time_ms}
   request_data = {"query": query, "timeframe": timeframe}
   if tenants is not None:
       request_data["tenants"] = tenants
   payload = {"request_data": request_data}
   resp = SESSION.post(url, data=json.dumps(payload))
   resp.raise_for_status()
   data = resp.json()
   reply = data.get("reply", data)
   if isinstance(reply, dict):
       return reply.get("query_id") or reply.get("execution_id")
   return reply

def get_query_results_basic(query_id, limit=1000000, pending_ok=False):
 
   url = f"{BASE_URL}/public_api/v1/xql/get_query_results"
   request_data = {"query_id": query_id, "format": "json"}
   if limit is not None:
       request_data["limit"] = limit
   payload = {"request_data": request_data}
   resp = SESSION.post(url, data=json.dumps(payload))
   resp.raise_for_status()
   data = resp.json()
   reply = data.get("reply", data)
   status = reply.get("status", "SUCCESS")
   results = reply.get("results", {})
   stream_id = results.get("stream_id")
   rows = []
   if "data" in results:
       rows = results["data"]
   elif "data" in reply:
       rows = reply["data"]
   df = pd.DataFrame(rows)
   if status != "SUCCESS" and not pending_ok:
       raise ValueError(f"Results not ready yet, status={status}\n{json.dumps(data, indent=2)[:1000]}")
   return data, df, status, stream_id

def download_stream_to_file(stream_id):
  
   
   timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S_%f")
   base_dir = os.path.expanduser(f"~/XQL_Results/{timestamp}")
   os.makedirs(base_dir, exist_ok=True)
   gz_path = os.path.join(base_dir, "results.json.gz")
   json_path = os.path.join(base_dir, "results.json")
   # Build request
   url = f"{BASE_URL}/public_api/v1/xql/get_query_results_stream"
   headers = {
       "Accept": "application/json",
       "Accept-Encoding": "gzip",
       "Authorization": API_KEY,
       "x-xdr-auth-id": API_KEY_ID,
       "Content-Type": "application/json"
   }
   body = {
       "request_data": {
           "stream_id": stream_id,
           "is_gzip_compressed": True
       }
   }
   
   with requests.post(url, headers=headers, json=body, stream=True) as r:
       r.raise_for_status()
       # Save gzip file to disk
       with open(gz_path, "wb") as f:
           for chunk in r.iter_content(chunk_size=8192):
               if chunk:
                   f.write(chunk)
   print(f"[OK] Gzip stream saved to: {gz_path}")
   with gzip.open(gz_path, "rb") as gz_file:
       data = gz_file.read()
   with open(json_path, "wb") as out_json:
       out_json.write(data)
   print(f"[OK] Extracted JSON to: {json_path}")
   return json_path

def load_json_rows(json_path):
  
   rows = []
   with open(json_path, "r", encoding="utf-8", errors="ignore") as f:
       for line in f:
           line = line.strip()
           if not line:
               continue
           if line in ["[", "]", "{", "}", ","]:
               continue
           try:
               obj = json.loads(line)
               rows.append(obj)
           except json.JSONDecodeError:
               continue
   print(f"[OK] Loaded {len(rows)} rows from JSON")
   return pd.DataFrame(rows)

def run_xql_auto_download(query):
 
   query_id = start_xql_query(query)
   print(f"[run_xql_auto] Started query_id={query_id}")
   # poll until SUCCESS
   for attempt in range(POLL_MAX_ATTEMPTS):
       data, df_basic, status, stream_id = get_query_results_basic(
           query_id=query_id,
           limit=1000000,
           pending_ok=True
       )
       print(f"[poll] attempt={attempt+1}, status={status}, stream_id={stream_id}")
       if status == "SUCCESS":
           break
       time.sleep(POLL_SLEEP_SECONDS)
   else:
       raise RuntimeError(f"Timed out polling XQL results for query {query_id}")
   
   if stream_id:
       print(f"[run_xql_auto] Large result set – downloading stream {stream_id}")
       json_path = download_stream_to_file(stream_id)

       global LAST_STREAM_JSON_PATH
       LAST_STREAM_JSON_PATH = json_path
       df = load_json_rows(json_path)
       print(f"[run_xql_auto] Final row count: {len(df)}")
       return df, LAST_STREAM_JSON_PATH
   
   print(f"[run_xql_auto] Using basic results: {len(df_basic)} rows")
   return df_basic, None

# ===========================================================
# XQL Query CELL
# ===========================================================
xql = """
#<Your XQL Query>
"""
df, _ = run_xql_auto_download(xql)
print(f"Total rows returned (auto): {len(df)}")
df.head(20)        # only print first 20 rows 
